In [1]:
import pandas as pd
import re

In [2]:
# read in te data from raw/external-data folder
df_high_school_rank = pd.read_csv('../../data/raw/external-data/school_clean.csv')
df_primary_school_rank = pd.read_csv('../../data/external-raw-data/primary_school_rank.csv')

In [4]:
# unified the type of the column postcode(originally exsits 1 column of float, and others are all strings)
df_high_school_rank = df_high_school_rank.astype({'postcode': str})

In [5]:
# filter out the schools with not found or nan in postcode column
for i in range(0, len(df_high_school_rank)):
    if df_high_school_rank['postcode'][i].isnumeric():
        continue
    else:
        df_high_school_rank.drop(i, inplace=True)

In [6]:
# create two dictionary to store how many schools in each postcode
high_dic_postcode = df_high_school_rank.groupby('postcode').count().to_dict()['School']
primary_dic_postcode = df_primary_school_rank.groupby('Postcode').count().to_dict()['School name']

In [7]:
# get the list postcode
high_lst_postcode = sorted(list(set(df_high_school_rank['postcode'])))
primary_lst_postcode = sorted(list(set(df_primary_school_rank['Postcode'])))

In [8]:
# sort the dataframes by poscode to reduce the number of comparision
df_sorted_high_school_rank = df_high_school_rank.sort_values(by=['postcode'], ascending=True, ignore_index = True)
df_sorted_primary_school_rank = df_primary_school_rank.sort_values(by=['Postcode'], ascending=True, ignore_index = True)

In [9]:
# create one dataframe and one for primary school to store the result: how many schools has the rank in each level
headers = ["POSTCODE", "1-10", "11-50", "51-100", "101-150", "150+"]
high_school = pd.DataFrame(columns = headers, index=range(len(high_lst_postcode)))
primary_school = pd.DataFrame(columns = headers, index=range(len(primary_lst_postcode)))

In [10]:
# define a fuction to fill in the dataframe
def education_resource(df_school, dic_postcode, empty_df):
    
    # index indicates which row is up to
    index = 0
    
    for j in range(0, len(dic_postcode.keys())):
        
        # get the postcode for this suburb
        postcode = list(dic_postcode.keys())[j]
        empty_df.loc[j]["POSTCODE"] = postcode
        
        # for each suburb, initialise the number of school in each level by 0
        empty_df.loc[j]["1-10"] = 0
        empty_df.loc[j]["11-50"] = 0 
        empty_df.loc[j]["51-100"] = 0 
        empty_df.loc[j]["101-150"] = 0
        empty_df.loc[j]["150+"] = 0 
        
        # count is the total number of schools in this suburb
        count = dic_postcode[postcode]
        
        # for each school in this suburb, check its rank and increase the number of school in the corresponding level
        for i in range(index, index+count):
            if df_school.loc[i]['rank'] <= 10:
                empty_df.loc[j]["1-10"] += 1
            if (df_school.loc[i]['rank'] > 10) & (df_school.loc[i]['rank'] <= 50):
                empty_df.loc[j]["11-50"] += 1
            if (df_school.loc[i]['rank'] > 50) & (df_school.loc[i]['rank'] <= 100):
                empty_df.loc[j]["51-100"] += 1  
            if (df_school.loc[i]['rank'] > 100) & (df_school.loc[i]['rank'] <= 150):
                empty_df.loc[j]["101-150"] += 1
            if (df_school.loc[i]['rank'] > 150):    
                empty_df.loc[j]["150+"] += 1
                
        # update the position pointer
        index = index+count
    return

In [11]:
# use fuction to fill the dataframes
education_resource(df_sorted_high_school_rank, high_dic_postcode, high_school)
education_resource(df_sorted_primary_school_rank, primary_dic_postcode, primary_school)

In [14]:
high_school.to_csv('../../data/raw/external-data/postcode_high_school.csv', index=False)
primary_school.to_csv('../../data/raw/external-data/postcode_primary_school.csv', index=False)